In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from tqdm import tqdm
from tensorflow.keras.models import load_model
from PIL import Image 

In [12]:
# Configuración básica
train_dir = 'C:/EPN/2024-B/IA/Proyecto/Datos/procesados2/train_balanced'
val_dir = 'C:/EPN/2024-B/IA/Proyecto/Datos/procesados2/val_balanced'
batch_size = 32
num_classes = 3  # Tonalidad blanca, tonalidad media, tonalidad oscura
learning_rate = 0.001
num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
# Transformaciones para preprocesar las imágenes
data_transforms = {
    'train': transforms.Compose([
        # Redimensiona las imágenes a 224x224 píxeles
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        # Transforma las imágenes a tensores de PyTorch
        transforms.ToTensor(),
        # Normalización de las imágenes con la media y desviación estándar de ImageNet.
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [7]:
# Cargar los datos
train_dataset = datasets.ImageFolder(train_dir, transform=data_transforms['train'])
val_dataset = datasets.ImageFolder(val_dir, transform=data_transforms['val'])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [8]:
modelo_cargado = torch.load('C:/EPN/2024-B/IA/Proyecto/Modelos/model2_resnet18.pth')
# Cambiar a modo evaluación si deseas usarlo para predicciones
print("Modelo completo cargado correctamente.")

Modelo completo cargado correctamente.


C:\Users\qkevi\AppData\Local\Temp\ipykernel_12452\1693318551.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  modelo_cargado = torch.load('C:/EPN/2024-B/IA/Proyecto/Model

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(modelo_cargado.parameters(), lr=learning_rate)

In [10]:

def train_model_completly(model, train_loader, val_loader, criterion, optimizer, num_epochs):
    train_loss_history = []
    val_loss_history = []
    train_acc_history = []
    val_acc_history = []
    
    # Almacenar etiquetas y predicciones para classification reports
    all_train_labels_epoch = []  # Etiquetas reales del train por época
    all_train_preds_epoch = []  # Predicciones del train por época
    all_val_labels_epoch = []   # Etiquetas reales del val por época
    all_val_preds_epoch = []    # Predicciones del val por época

    for epoch in range(num_epochs):
        # Variables para acumular métricas del conjunto de entrenamiento
        all_train_labels = []
        all_train_preds = []

        # Entrenamiento
        model.train()
        running_loss = 0.0
        running_corrects = 0
        processed_images = 0

        # Barra de progreso para entrenamiento
        progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', leave=False)

        for inputs, labels in progress_bar:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            _, preds = torch.max(outputs, 1)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
            processed_images += inputs.size(0)

            # Acumular etiquetas reales y predicciones del entrenamiento
            all_train_labels.extend(labels.cpu().numpy())
            all_train_preds.extend(preds.cpu().numpy())

            # Actualizar la barra de progreso con la pérdida promedio
            progress_bar.set_postfix({'loss': running_loss / processed_images})

        # Calcular métricas del entrenamiento
        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = running_corrects.double() / len(train_loader.dataset)
        train_loss_history.append(epoch_loss)
        train_acc_history.append(epoch_acc.item())
        all_train_labels_epoch.append(all_train_labels)
        all_train_preds_epoch.append(all_train_preds)

        # Validación al final de cada época
        model.eval()
        val_loss = 0.0
        val_corrects = 0.0

        all_val_labels = []
        all_val_preds = []

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                _, preds = torch.max(outputs, 1)
                val_loss += loss.item() * inputs.size(0)
                val_corrects += torch.sum(preds == labels.data)

                # Acumular etiquetas reales y predicciones del conjunto de validación
                all_val_labels.extend(labels.cpu().numpy())
                all_val_preds.extend(preds.cpu().numpy())

        # Calcular métricas de validación
        val_loss /= len(val_loader.dataset)
        val_acc = val_corrects.double() / len(val_loader.dataset)
        val_loss_history.append(val_loss)
        val_acc_history.append(val_acc.item())
        all_val_labels_epoch.append(all_val_labels)
        all_val_preds_epoch.append(all_val_preds)

        # Imprimir métricas al final de la época
        print(f"Epoch {epoch+1}/{num_epochs} -> Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.4f}, "
              f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

    # Devolver todas las métricas y datos para los classification reports
    return (model, train_loss_history, train_acc_history, val_loss_history, val_acc_history, 
            all_train_labels_epoch, all_train_preds_epoch, all_val_labels_epoch, all_val_preds_epoch)


In [13]:
nuevoModelo, train_loss_history, train_acc_history, val_loss_history, val_acc_history, labels_all_train_epoch, labels_preds_train, labels_all_test_epoch, labels_test_preds = train_model_completly(modelo_cargado, train_loader, val_loader, criterion, optimizer, num_epochs)

Epoch 1/10 -> Train Loss: 0.0923, Train Acc: 0.9648, Val Loss: 0.0769, Val Acc: 0.9706


Epoch 2/10 -> Train Loss: 0.0925, Train Acc: 0.9643, Val Loss: 0.0669, Val Acc: 0.9733


Epoch 3/10 -> Train Loss: 0.0886, Train Acc: 0.9658, Val Loss: 0.0776, Val Acc: 0.9729


Epoch 4/10 -> Train Loss: 0.0821, Train Acc: 0.9681, Val Loss: 0.0714, Val Acc: 0.9733


Epoch 5/10 -> Train Loss: 0.0818, Train Acc: 0.9689, Val Loss: 0.0616, Val Acc: 0.9761


Epoch 6/10 -> Train Loss: 0.0788, Train Acc: 0.9689, Val Loss: 0.0624, Val Acc: 0.9737


Epoch 7/10 -> Train Loss: 0.0764, Train Acc: 0.9702, Val Loss: 0.0681, Val Acc: 0.9745


Epoch 8/10 -> Train Loss: 0.0759, Train Acc: 0.9708, Val Loss: 0.0589, Val Acc: 0.9761


Epoch 9/10 -> Train Loss: 0.0726, Train Acc: 0.9703, Val Loss: 0.0553, Val Acc: 0.9761


Epoch 10/10 -> Train Loss: 0.0679, Train Acc: 0.9737, Val Loss: 0.0752, Val Acc: 0.9749


In [14]:
torch.save(nuevoModelo, 'C:/EPN/2024-B/IA/Proyecto/Modelos/model3_resnet18.pth')
print("Modelo completo guardado correctamente.")

Modelo completo guardado correctamente.
